# Model Development and Comparison Demo

This notebook demonstrates the model development pipeline for financial time series prediction using wavelet features.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import our modules
import sys
sys.path.append('..')

from src.models.sequence_predictor import LSTMModel, GRUModel, TimeSeriesDataset, train_sequence_model
from src.models.transformer_predictor import TransformerPredictor
from src.models.xgboost_predictor import XGBoostPredictor, XGBoostTimeSeriesCV
from src.models.ensemble_model import EnsembleModel
from src.models.model_trainer import ModelTrainer, WalkForwardValidator
from src.models.model_evaluator import ModelEvaluator, create_evaluation_report

import torch
from torch.utils.data import DataLoader

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 1. Generate Synthetic Financial Data

In [ ]:
# Generate synthetic financial time series
np.random.seed(42)
n_samples = 2000
n_features = 50  # Simulating wavelet and technical features

# Time index
dates = pd.date_range(start='2018-01-01', periods=n_samples, freq='D')

# Generate base price series with trend, seasonality, and noise
trend = np.linspace(100, 150, n_samples)
seasonal = 10 * np.sin(2 * np.pi * np.arange(n_samples) / 365.25)
noise = 2 * np.random.randn(n_samples)
price = trend + seasonal + noise

# Add some market regime changes
regime_change_points = [500, 1000, 1500]
for cp in regime_change_points:
    price[cp:] += np.random.randn() * 5

# Calculate returns (target variable)
returns = np.diff(price) / price[:-1]
returns = np.concatenate([[0], returns])  # Pad first return

# Generate features (simulating wavelet coefficients and technical indicators)
features = np.random.randn(n_samples, n_features)

# Add some structure to features
# Wavelet-like features at different scales
for i in range(5):
    scale = 2 ** (i + 1)
    features[:, i] = np.convolve(returns, np.ones(scale)/scale, mode='same')

# Technical indicator-like features
features[:, 5] = pd.Series(price).rolling(20).mean().fillna(method='bfill').values  # MA20
features[:, 6] = pd.Series(price).rolling(50).mean().fillna(method='bfill').values  # MA50
features[:, 7] = pd.Series(returns).rolling(20).std().fillna(method='bfill').values  # Volatility

# Normalize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features = scaler.fit_transform(features)

print(f"Generated {n_samples} samples with {n_features} features")
print(f"Price range: ${price.min():.2f} - ${price.max():.2f}")
print(f"Return statistics: mean={returns.mean():.4f}, std={returns.std():.4f}")

In [ ]:
# Visualize the data
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Price series
axes[0].plot(dates, price, label='Price', color='blue', alpha=0.7)
axes[0].set_ylabel('Price ($)')
axes[0].set_title('Synthetic Financial Time Series')
axes[0].legend()

# Returns
axes[1].plot(dates, returns, label='Returns', color='green', alpha=0.7)
axes[1].set_ylabel('Returns')
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1].legend()

# Feature heatmap (first 10 features)
im = axes[2].imshow(features[:200, :10].T, aspect='auto', cmap='coolwarm')
axes[2].set_ylabel('Features')
axes[2].set_xlabel('Time')
axes[2].set_title('Feature Heatmap (First 10 Features, 200 Time Steps)')
plt.colorbar(im, ax=axes[2])

plt.tight_layout()
plt.show()

## 2. Data Preparation

In [ ]:
# Split data into train, validation, and test sets
train_size = int(0.7 * n_samples)
val_size = int(0.15 * n_samples)
test_size = n_samples - train_size - val_size

# Features and targets
X = features
y = returns

# Split
X_train = X[:train_size]
y_train = y[:train_size]
X_val = X[train_size:train_size+val_size]
y_val = y[train_size:train_size+val_size]
X_test = X[train_size+val_size:]
y_test = y[train_size+val_size:]

print(f"Train: {len(X_train)} samples")
print(f"Validation: {len(X_val)} samples")
print(f"Test: {len(X_test)} samples")

In [ ]:
# Prepare sequences for neural networks
def prepare_sequences(X, y, seq_length=20):
    X_seq = []
    y_seq = []
    
    for i in range(seq_length, len(X)):
        X_seq.append(X[i-seq_length:i])
        y_seq.append(y[i])
    
    return np.array(X_seq), np.array(y_seq)

# Create sequences
seq_length = 20
X_train_seq, y_train_seq = prepare_sequences(X_train, y_train, seq_length)
X_val_seq, y_val_seq = prepare_sequences(X_val, y_val, seq_length)
X_test_seq, y_test_seq = prepare_sequences(X_test, y_test, seq_length)

print(f"Sequence shape: {X_train_seq.shape}")
print(f"Target shape: {y_train_seq.shape}")

## 3. Train Individual Models

### 3.1 LSTM Model

In [ ]:
# Create LSTM model
lstm_model = LSTMModel(
    input_size=n_features,
    hidden_size=64,
    num_layers=2,
    dropout=0.2
)

print(f"LSTM Model Parameters: {lstm_model.count_parameters():,}")

# Create data loaders
train_dataset = TimeSeriesDataset(X_train_seq, y_train_seq)
val_dataset = TimeSeriesDataset(X_val_seq, y_val_seq)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Train model
print("\nTraining LSTM...")
lstm_history = train_sequence_model(
    lstm_model, train_loader, val_loader,
    epochs=50,
    learning_rate=0.001,
    early_stopping_patience=10
)

### 3.2 GRU Model

In [ ]:
# Create GRU model
gru_model = GRUModel(
    input_size=n_features,
    hidden_size=64,
    num_layers=2,
    dropout=0.2
)

print(f"GRU Model Parameters: {gru_model.count_parameters():,}")

# Train model
print("\nTraining GRU...")
gru_history = train_sequence_model(
    gru_model, train_loader, val_loader,
    epochs=50,
    learning_rate=0.001,
    early_stopping_patience=10
)

### 3.3 Transformer Model

In [ ]:
# Create Transformer model
transformer_model = TransformerPredictor(
    input_size=n_features,
    d_model=64,
    n_heads=4,
    n_layers=2,
    d_ff=256,
    dropout=0.1,
    max_seq_len=seq_length
)

print(f"Transformer Model Parameters: {transformer_model.count_parameters():,}")

# Train model
print("\nTraining Transformer...")
transformer_history = train_sequence_model(
    transformer_model, train_loader, val_loader,
    epochs=50,
    learning_rate=0.001,
    early_stopping_patience=10
)

### 3.4 XGBoost Model

In [ ]:
# Create XGBoost model
xgb_model = XGBoostPredictor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8
)

# Train model
print("Training XGBoost...")
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

print(f"Best iteration: {xgb_model.best_iteration_}")

## 4. Compare Training Histories

In [ ]:
# Plot training histories
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Training loss
axes[0].plot(lstm_history['train_loss'], label='LSTM', alpha=0.7)
axes[0].plot(gru_history['train_loss'], label='GRU', alpha=0.7)
axes[0].plot(transformer_history['train_loss'], label='Transformer', alpha=0.7)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].set_yscale('log')

# Validation loss
axes[1].plot(lstm_history['val_loss'], label='LSTM', alpha=0.7)
axes[1].plot(gru_history['val_loss'], label='GRU', alpha=0.7)
axes[1].plot(transformer_history['val_loss'], label='Transformer', alpha=0.7)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Validation Loss')
axes[1].legend()
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

## 5. Model Evaluation

In [ ]:
# Make predictions for all models
models = {
    'LSTM': (lstm_model, X_test_seq),
    'GRU': (gru_model, X_test_seq),
    'Transformer': (transformer_model, X_test_seq),
    'XGBoost': (xgb_model, X_test)
}

predictions = {}
for name, (model, X_data) in models.items():
    if name == 'XGBoost':
        predictions[name] = model.predict(X_data)
    else:
        model.eval()
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X_data)
            pred = model(X_tensor).numpy().flatten()
            predictions[name] = pred

# Align test data for comparison
y_test_aligned = y_test_seq if 'LSTM' in predictions else y_test

In [ ]:
# Evaluate all models
evaluator = ModelEvaluator()
results = {}

for name, y_pred in predictions.items():
    # Use appropriate test set
    y_true = y_test_seq if name != 'XGBoost' else y_test
    
    metrics = evaluator.evaluate(y_true, y_pred)
    results[name] = metrics
    
    print(f"\n{name} Results:")
    print(f"  RMSE: {metrics['rmse']:.6f}")
    print(f"  MAE: {metrics['mae']:.6f}")
    print(f"  R²: {metrics['r2']:.4f}")
    print(f"  Direction Accuracy: {metrics.get('directional_accuracy', 'N/A')}")

In [ ]:
# Create comparison visualization
results_df = pd.DataFrame(results).T
metrics_to_plot = ['rmse', 'mae', 'r2', 'directional_accuracy']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for i, metric in enumerate(metrics_to_plot):
    ax = axes[i]
    values = results_df[metric].values
    models_list = results_df.index
    
    bars = ax.bar(models_list, values, alpha=0.7)
    
    # Color bars based on performance
    if metric in ['rmse', 'mae']:  # Lower is better
        best_idx = np.argmin(values)
    else:  # Higher is better
        best_idx = np.argmax(values)
    
    bars[best_idx].set_color('green')
    
    ax.set_title(f'{metric.upper()}')
    ax.set_ylabel(metric.upper())
    
    # Add value labels
    for j, v in enumerate(values):
        ax.text(j, v + 0.001 * max(values), f'{v:.4f}', 
                ha='center', va='bottom')

plt.suptitle('Model Performance Comparison', fontsize=16)
plt.tight_layout()
plt.show()

## 6. Ensemble Model

In [ ]:
# Create ensemble with XGBoost and a simple Ridge model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

# Train additional models for ensemble
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

rf_model = RandomForestRegressor(n_estimators=50, random_state=42)
rf_model.fit(X_train, y_train)

# Create ensemble
ensemble_models = {
    'XGBoost': xgb_model,
    'Ridge': ridge_model,
    'RandomForest': rf_model
}

# Test different ensemble strategies
ensemble_strategies = ['averaging', 'median', 'stacking']
ensemble_results = {}

for strategy in ensemble_strategies:
    print(f"\nTesting {strategy} ensemble...")
    
    if strategy == 'stacking':
        ensemble = EnsembleModel(
            models=ensemble_models.copy(),
            strategy=strategy,
            meta_learner=Ridge(alpha=0.1)
        )
    else:
        ensemble = EnsembleModel(
            models=ensemble_models.copy(),
            strategy=strategy
        )
    
    # Fit ensemble
    ensemble.fit(X_train, y_train)
    
    # Evaluate
    eval_results = ensemble.evaluate(X_test, y_test)
    ensemble_results[f'Ensemble_{strategy}'] = eval_results['ensemble']
    
    print(f"Ensemble ({strategy}) RMSE: {eval_results['ensemble']['rmse']:.6f}")

## 7. Feature Importance Analysis

In [ ]:
# XGBoost feature importance
importance = xgb_model.get_feature_importance()
importance_values = list(importance.values())
importance_indices = np.argsort(importance_values)[-20:]  # Top 20

plt.figure(figsize=(10, 8))
plt.barh(range(len(importance_indices)), 
         [importance_values[i] for i in importance_indices],
         alpha=0.7)
plt.yticks(range(len(importance_indices)), 
           [f'Feature {i}' for i in importance_indices])
plt.xlabel('Importance Score')
plt.title('Top 20 Feature Importances (XGBoost)')
plt.tight_layout()
plt.show()

## 8. Prediction Visualization

In [ ]:
# Visualize predictions for a subset of test data
n_plot = 100
plot_start = 0
plot_end = plot_start + n_plot

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i, (name, y_pred) in enumerate(predictions.items()):
    if i >= 4:
        break
        
    ax = axes[i]
    
    # Use appropriate test set
    y_true = y_test_seq if name != 'XGBoost' else y_test
    
    # Plot subset
    x_range = range(plot_start, min(plot_end, len(y_true)))
    ax.plot(x_range, y_true[plot_start:plot_end], 
            label='Actual', alpha=0.7, linewidth=2)
    ax.plot(x_range, y_pred[plot_start:plot_end], 
            label='Predicted', alpha=0.7, linewidth=2)
    
    ax.set_title(f'{name} Predictions')
    ax.set_xlabel('Time')
    ax.set_ylabel('Returns')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Model Predictions vs Actual Returns', fontsize=16)
plt.tight_layout()
plt.show()

## 9. Walk-Forward Validation

In [ ]:
# Perform walk-forward validation for XGBoost
print("Performing walk-forward validation...")

validator = WalkForwardValidator(
    train_size=1000,
    test_size=100,
    step_size=50,
    retrain_frequency=4
)

# Create trainer for walk-forward
xgb_trainer = ModelTrainer(
    model_type='xgboost',
    model_config={
        'n_estimators': 50,
        'max_depth': 4,
        'learning_rate': 0.1
    },
    training_config={},
    use_mlflow=False
)

# Run walk-forward validation
wf_results = validator.validate(xgb_trainer, X, y)

print(f"\nWalk-forward validation results:")
print(f"Overall RMSE: {wf_results['overall_metrics']['rmse']:.6f}")
print(f"Overall MAE: {wf_results['overall_metrics']['mae']:.6f}")
print(f"Overall R²: {wf_results['overall_metrics']['r2']:.4f}")

In [ ]:
# Plot walk-forward results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Predictions vs Actuals
axes[0, 0].scatter(wf_results['actuals'], wf_results['predictions'], 
                   alpha=0.5, s=10)
axes[0, 0].plot([min(wf_results['actuals']), max(wf_results['actuals'])],
                [min(wf_results['actuals']), max(wf_results['actuals'])],
                'r--', lw=2)
axes[0, 0].set_xlabel('Actual')
axes[0, 0].set_ylabel('Predicted')
axes[0, 0].set_title('Walk-Forward: Predictions vs Actuals')

# Time series of predictions
axes[0, 1].plot(wf_results['actuals'], label='Actual', alpha=0.7)
axes[0, 1].plot(wf_results['predictions'], label='Predicted', alpha=0.7)
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Value')
axes[0, 1].set_title('Walk-Forward: Time Series')
axes[0, 1].legend()

# Rolling window metrics
window_mse = [m['mse'] for m in wf_results['metrics']]
window_starts = [m['window_start'] for m in wf_results['metrics']]
axes[1, 0].plot(window_starts, window_mse, marker='o', alpha=0.7)
axes[1, 0].set_xlabel('Window Start')
axes[1, 0].set_ylabel('MSE')
axes[1, 0].set_title('Rolling Window MSE')
axes[1, 0].grid(True, alpha=0.3)

# Residuals
residuals = np.array(wf_results['actuals']) - np.array(wf_results['predictions'])
axes[1, 1].hist(residuals, bins=50, alpha=0.7, edgecolor='black')
axes[1, 1].axvline(x=0, color='red', linestyle='--')
axes[1, 1].set_xlabel('Residual')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Residual Distribution')

plt.suptitle('Walk-Forward Validation Results', fontsize=16)
plt.tight_layout()
plt.show()

## 10. Summary and Conclusions

In [ ]:
# Create final summary
all_results = {**results, **ensemble_results}
summary_df = pd.DataFrame(all_results).T[['rmse', 'mae', 'r2', 'directional_accuracy']]
summary_df = summary_df.sort_values('rmse')

print("\n" + "="*60)
print("FINAL MODEL COMPARISON SUMMARY")
print("="*60)
print("\nModels ranked by RMSE (lower is better):")
print(summary_df.to_string())

# Best model
best_model = summary_df.index[0]
print(f"\nBest performing model: {best_model}")
print(f"  RMSE: {summary_df.loc[best_model, 'rmse']:.6f}")
print(f"  R²: {summary_df.loc[best_model, 'r2']:.4f}")
print(f"  Direction Accuracy: {summary_df.loc[best_model, 'directional_accuracy']:.4f}")